<br>

# DAEE

In [1]:
import os
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
def predict_encoding(file_path, n_lines=30):
    """
    Predict Encoding determines the encoding of a given file, so that when reading the file (via 'Pandas', for example)
    characters, accents and symbols are spelled correctly.
    :param file_path: Name and path to file
    :param n_lines: Number of lines to read to set encoding. Parameter Optional. Default: 30 lines.
    :return: Encoding.
    """
    import chardet

    # Open the file as binary data
    with open(file_path, 'rb') as f:
        # Join binary lines for specified number of lines
        raw_data = b''.join([f.readline() for line in range(n_lines)])

    return chardet.detect(raw_data)['encoding']

<br>

# Read Data

In [3]:
my_file = os.path.join('data', 'brutos', 'daee_arq8.txt')

enc = predict_encoding(my_file, n_lines=300)
enc

'ascii'

In [4]:
df = pd.read_csv(
    os.path.join('data', 'brutos', 'daee_arq8.txt'),
    sep=';',
    skiprows=1,
    #encoding=enc,
    dtype={
        # Identificação
        #'QTD': 'quantidade',
        #'COD BACIA': 'cod_bacia',
        'NOME RIO/AQUIFERO': 'object',
        'COD RIO/POCO(DAEE)': 'object',    
        'DIST_FOZ(Km)': 'object',

        # Administrativo
        #'USUARIO': 'usuario',    
        #'AUTOS(DAEE)': 'autos_daee',
        #'USO': 'uso',
        #'SEQ': 'seq',
        #'FINALID_USO': 'finalidade_uso',
        #'SITUAC_ADMIN(DAEE)': 'situacao_administrativa',

        # Vazão
        #'VAZAO(M3/H)': 'vazao_m3h',
        #'HORA/DIA': 'hora_dia',
        #'DIA/MES': 'dia_mes',
        #'MES/ANO': 'mes_ano',

        # Coordenada
        'UTM_NORTE(Km)': 'object',
        'UTM_LESTE(Km)': 'object',
        'UTM_MC': 'object',
    }
)

df.dtypes

QTD                    int64
COD BACIA              int64
NOME RIO/AQUIFERO     object
COD RIO/POCO(DAEE)    object
DIST_FOZ(Km)          object
USUARIO               object
AUTOS(DAEE)            int64
USO                   object
SEQ                    int64
FINALID_USO           object
SITUAC_ADMIN(DAEE)    object
VAZAO(M3/H)           object
HORA/DIA               int64
DIA/MES                int64
MES/ANO                int64
UTM_NORTE(Km)         object
UTM_LESTE(Km)         object
UTM_MC                object
dtype: object

In [5]:
# Colunas Originiais
#display(list(df.columns))

# Renomear Colunas
dict_columns = {
    # Identificação
    'QTD': 'quantidade',
    'COD BACIA': 'cod_bacia',
    'NOME RIO/AQUIFERO': 'nome_rio_aquifero',
    'COD RIO/POCO(DAEE)': 'cod_rio_poco',    
    'DIST_FOZ(Km)': 'dist_foz_km',
    
    # Administrativo
    'USUARIO': 'usuario',
    'AUTOS(DAEE)': 'autos_daee',
    'USO': 'uso',
    'SEQ': 'seq',
    'FINALID_USO': 'finalidade_uso',
    'SITUAC_ADMIN(DAEE)': 'situacao_administrativa',
    
    # Vazão
    'VAZAO(M3/H)': 'vazao_m3h',
    'HORA/DIA': 'hora_dia',
    'DIA/MES': 'dia_mes',
    'MES/ANO': 'mes_ano',
    
    # Coordenada
    'UTM_NORTE(Km)': 'utm_norte_km',
    'UTM_LESTE(Km)': 'utm_leste_km',
    'UTM_MC': 'utm_mc',
}

# Rename Columns
df.rename(columns=dict_columns, inplace=True, errors='ignore')
df

,quantidade,cod_bacia,nome_rio_aquifero,cod_rio_poco,dist_foz_km,usuario,autos_daee,uso,seq,finalidade_uso,situacao_administrativa,vazao_m3h,hora_dia,dia_mes,mes_ano,utm_norte_km,utm_leste_km,utm_mc
0,1,5,...,,0.00,PUBLICO,35010,CAPTACAO SUBTERRANEA,1,,,10.00,12,0,0,0.00,0.00,0
1,2,6,...,,0.00,PUBLICO,34843,LANCAMENTO SUPERFICIAL,1,,,0.00,0,0,0,0.00,0.00,0
2,3,8,...,,0.00,PUBLICO,9200002,CAPTACAO SUBTERRANEA,1,AB.PUBL,LICENCA OPERACAO,200.00,20,0,0,0.00,0.00,0
3,4,5,...,,0.00,PUBLICO,9701259,CAPTACAO SUBTERRANEA,1,,,648.00,20,0,0,0.00,0.00,0
4,5,5,FORMACAO ADAMANTINA ...,,0.00,PUBLICO,9405973,CAPTACAO SUBTERRANEA,1,AB.PUBL,LIC PERFURACAO,0.00,0,0,0,"7,622.30",570.20,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188249,188250,6,...,R-3,0.00,AQUICULTOR,9407255,BARRAMENTO,3,HIDROAG,REQ INDEFERIDO,0.00,0,0,0,"7,395.44",681.63,51
188250,188251,2,...,S10201817,3.80,PUBLICO,9800142,TRAVESSIA INTERMEDIARIA,4,PASDUTO,PORTARIA,0.00,0,0,0,"7,395.94",247.07,45
188251,188252,3,CRISTALINO ...,S840057,0.00,PUBLICO,9603261,CAPTACAO SUBTERRANEA,7,AB.PUBL,PORTARIA,16.80,16,30,0,"7,480.54",473.48,45
188252,188253,2,...,VALID.: PRAZO DE 3,6.81,US.RURAL,9814008,TRAVESSIA AEREA,1,PASSAGE,IMPL AUTORIZADA,0.00,0,0,0,"7,479.30",311.36,45


In [6]:
# Drop Columns
df.drop('quantidade', axis=1, inplace=True, errors='ignore')

# Distância da Foz
df['dist_foz_km'] = df['dist_foz_km'].str.replace(',', '').astype(float)
df['dist_foz_km'] = pd.to_numeric(df['dist_foz_km'])

# Vazão
df['vazao_m3h'] = df['vazao_m3h'].str.replace(',', '').astype(float)
df['vazao_m3h'] = pd.to_numeric(df['vazao_m3h'])

# UTM Norte
df['utm_norte_km'] = df['utm_norte_km'].str.replace(',', '').astype(float)
df['utm_norte_km'] = pd.to_numeric(df['utm_norte_km'])
df['utm_norte'] = df['utm_norte_km'] * 1000
df['utm_norte'] = df['utm_norte'].astype(int)

# UTM Leste
df['utm_leste_km'] = df['utm_leste_km'].str.replace(',', '').astype(float)
df['utm_leste_km'] = pd.to_numeric(df['utm_leste_km'])
df['utm_leste'] = df['utm_leste_km'] * 1000
df['utm_leste'] = df['utm_leste'].astype(int)

# UTM MC
df['utm_mc'] = df['utm_mc'].str.replace(',', '').astype(float)
df['utm_mc'] = pd.to_numeric(df['utm_mc'])
df['utm_mc'] = df['utm_mc'].astype(int)

# Results
df

,cod_bacia,nome_rio_aquifero,cod_rio_poco,dist_foz_km,usuario,autos_daee,uso,seq,finalidade_uso,situacao_administrativa,vazao_m3h,hora_dia,dia_mes,mes_ano,utm_norte_km,utm_leste_km,utm_mc,utm_norte,utm_leste
0,5,...,,0.00,PUBLICO,35010,CAPTACAO SUBTERRANEA,1,,,10.0,12,0,0,0.00,0.00,0,0,0
1,6,...,,0.00,PUBLICO,34843,LANCAMENTO SUPERFICIAL,1,,,0.0,0,0,0,0.00,0.00,0,0,0
2,8,...,,0.00,PUBLICO,9200002,CAPTACAO SUBTERRANEA,1,AB.PUBL,LICENCA OPERACAO,200.0,20,0,0,0.00,0.00,0,0,0
3,5,...,,0.00,PUBLICO,9701259,CAPTACAO SUBTERRANEA,1,,,648.0,20,0,0,0.00,0.00,0,0,0
4,5,FORMACAO ADAMANTINA ...,,0.00,PUBLICO,9405973,CAPTACAO SUBTERRANEA,1,AB.PUBL,LIC PERFURACAO,0.0,0,0,0,7622.30,570.20,51,7622300,570200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188249,6,...,R-3,0.00,AQUICULTOR,9407255,BARRAMENTO,3,HIDROAG,REQ INDEFERIDO,0.0,0,0,0,7395.44,681.63,51,7395440,681630
188250,2,...,S10201817,3.80,PUBLICO,9800142,TRAVESSIA INTERMEDIARIA,4,PASDUTO,PORTARIA,0.0,0,0,0,7395.94,247.07,45,7395940,247070
188251,3,CRISTALINO ...,S840057,0.00,PUBLICO,9603261,CAPTACAO SUBTERRANEA,7,AB.PUBL,PORTARIA,16.8,16,30,0,7480.54,473.48,45,7480540,473480
188252,2,...,VALID.: PRAZO DE 3,6.81,US.RURAL,9814008,TRAVESSIA AEREA,1,PASSAGE,IMPL AUTORIZADA,0.0,0,0,0,7479.30,311.36,45,7479300,311360


In [7]:
list_columns = [
    'cod_bacia',
    #'nome_rio_aquifero',
    #'cod_rio_poco',
    #'dist_foz_km',
    #'usuario',
    #'autos_daee',
    #'uso',
    #'seq',
    #'finalidade_uso',
    #'situacao_administrativa',
    #'vazao_m3h',
    #'hora_dia',
    #'dia_mes',
    #'mes_ano',
    #'utm_norte_km',
    #'utm_leste_km',
    #'utm_mc',
    #'utm_norte',
    #'utm_leste'
]

for col in list_columns:
    try:
        df[col] = df[col].str.strip()
    except:
        pass
    print('>>> {}'.format(col))
    #display(set(df[col]))

>>> cod_bacia


<br>

# Erros

In [8]:
# Erro: Horas por Dia maior que 24
mask = df['hora_dia'] <= 24
mask =~ mask
df.loc[mask, 'erro_data'] = 'Erro na hora/dia/mês/ano'

# Erro: Dias por mês maior que 31
mask = df['dia_mes']  <= 31
mask =~ mask
df.loc[mask, 'erro_data'] = 'Erro na hora/dia/mês/ano'

# Erro: Mês por Ano 12
mask = df['mes_ano']  <= 12
mask =~ mask
df.loc[mask, 'erro_data'] = 'Erro na hora/dia/mês/ano'

# Erro: Coordenadas sem definição
mask = df['utm_leste_km'] == 0
df.loc[mask, 'erro_coordenada'] = 'Erro na coordenada'

# Erro: Coordenadas sem definição
mask = df['utm_norte_km'] == 0
df.loc[mask, 'erro_coordenada'] = 'Erro na coordenada'

# Erro: Coordenadas sem definição de Meridicano Central
mask = df['utm_mc'] == 0
df.loc[mask, 'erro_coordenada'] = 'Erro na coordenada'

# Results
df

,cod_bacia,nome_rio_aquifero,cod_rio_poco,dist_foz_km,usuario,autos_daee,uso,seq,finalidade_uso,situacao_administrativa,...,hora_dia,dia_mes,mes_ano,utm_norte_km,utm_leste_km,utm_mc,utm_norte,utm_leste,erro_data,erro_coordenada
0,5,...,,0.00,PUBLICO,35010,CAPTACAO SUBTERRANEA,1,,,...,12,0,0,0.00,0.00,0,0,0,NaN,Erro na coordenada
1,6,...,,0.00,PUBLICO,34843,LANCAMENTO SUPERFICIAL,1,,,...,0,0,0,0.00,0.00,0,0,0,NaN,Erro na coordenada
2,8,...,,0.00,PUBLICO,9200002,CAPTACAO SUBTERRANEA,1,AB.PUBL,LICENCA OPERACAO,...,20,0,0,0.00,0.00,0,0,0,NaN,Erro na coordenada
3,5,...,,0.00,PUBLICO,9701259,CAPTACAO SUBTERRANEA,1,,,...,20,0,0,0.00,0.00,0,0,0,NaN,Erro na coordenada
4,5,FORMACAO ADAMANTINA ...,,0.00,PUBLICO,9405973,CAPTACAO SUBTERRANEA,1,AB.PUBL,LIC PERFURACAO,...,0,0,0,7622.30,570.20,51,7622300,570200,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188249,6,...,R-3,0.00,AQUICULTOR,9407255,BARRAMENTO,3,HIDROAG,REQ INDEFERIDO,...,0,0,0,7395.44,681.63,51,7395440,681630,NaN,NaN
188250,2,...,S10201817,3.80,PUBLICO,9800142,TRAVESSIA INTERMEDIARIA,4,PASDUTO,PORTARIA,...,0,0,0,7395.94,247.07,45,7395940,247070,NaN,NaN
188251,3,CRISTALINO ...,S840057,0.00,PUBLICO,9603261,CAPTACAO SUBTERRANEA,7,AB.PUBL,PORTARIA,...,16,30,0,7480.54,473.48,45,7480540,473480,NaN,NaN
188252,2,...,VALID.: PRAZO DE 3,6.81,US.RURAL,9814008,TRAVESSIA AEREA,1,PASSAGE,IMPL AUTORIZADA,...,0,0,0,7479.30,311.36,45,7479300,311360,NaN,NaN


<br>

# Export

In [9]:
df.to_csv(
    os.path.join('data', 'tab_daee.csv'),
    index=False,
)

In [10]:
# Exclui Problemas
df = df[df['erro_data'].isna()]
df = df[df['erro_coordenada'].isna()]

# Export Clean
df.to_csv(
    os.path.join('data', 'tab_daee_clean.csv'),
    index=False,
)